In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import cv2
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.models import Sequential
from sklearn.metrics import accuracy_score
import os

Using TensorFlow backend.


In [5]:
%%time
#Чтобы не терять время на препроцессинг, просто загрузим ранее приготовленные данные
training_fruit_img = np.load('training_fruit_img.npy')
test_fruit_img = np.load('test_fruit_img.npy')
training_label_id = np.load('training_label_id.npy')
test_label_id = np.load('test_label_id.npy')

Wall time: 19.3 s


In [4]:
#Исходная архетиктура сети, состоящая из 4х свёрточных слоёв и двух полносвязных.
model = Sequential()
model.add(Conv2D(16, (3, 3), input_shape = (64, 64, 3), padding = "same", activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3), padding = "same", activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3), padding = "same", activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, (3, 3), padding = "same", activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dense(120, activation = "softmax"))

model.compile(loss = "sparse_categorical_crossentropy", optimizer = 'Adam', metrics = ['accuracy'])

In [6]:
%%time
model.fit(training_fruit_img, training_label_id, batch_size = 256, epochs = 10)

Epoch 1/10
60498/60498 [==============================] - 10s 170us/step - loss: 1.2237 - accuracy: 0.6970
Epoch 2/10
60498/60498 [==============================] - 8s 128us/step - loss: 0.0938 - accuracy: 0.9735
Epoch 3/10
60498/60498 [==============================] - 8s 128us/step - loss: 0.1441 - accuracy: 0.9652
Epoch 4/10
60498/60498 [==============================] - 8s 128us/step - loss: 0.0131 - accuracy: 0.9974
Epoch 5/10
60498/60498 [==============================] - 8s 128us/step - loss: 0.0071 - accuracy: 0.9989
Epoch 6/10
60498/60498 [==============================] - 8s 128us/step - loss: 0.0024 - accuracy: 0.9997
Epoch 7/10
60498/60498 [==============================] - 8s 128us/step - loss: 0.0066 - accuracy: 0.9984
Epoch 8/10
60498/60498 [==============================] - 8s 128us/step - loss: 0.0561 - accuracy: 0.9846
Epoch 9/10
60498/60498 [==============================] - 8s 128us/step - loss: 0.0014 - accuracy: 0.9998
Epoch 10/10
60498/60498 [====================

In [7]:
#Валидация на тесте
predicted_classes = model.predict_classes(test_fruit_img)
accuracy_score(predicted_classes, test_label_id)

0.9755600814663951

In [68]:
#Попробуем использовать свёрточные слои предобученной сети VGG16
from keras.applications import VGG16
image_size = 64
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))
for layer in vgg_conv.layers[:-2]:
    layer.trainable = False

model = Sequential()
model.add(vgg_conv)
model.add(Flatten())
model.add(Dense(128, activation='relu', )) 
model.add(Dense(120, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [69]:
%%time
model.fit(training_fruit_img, training_label_id, batch_size = 256, epochs = 10)

Epoch 1/10
60498/60498 [==============================] - 25s 408us/step - loss: 0.6339 - accuracy: 0.8620
Epoch 2/10
60498/60498 [==============================] - 25s 405us/step - loss: 0.0119 - accuracy: 0.9989
Epoch 3/10
60498/60498 [==============================] - 25s 405us/step - loss: 0.0032 - accuracy: 0.9999
Epoch 4/10
60498/60498 [==============================] - 25s 406us/step - loss: 0.0139 - accuracy: 0.9966
Epoch 5/10
60498/60498 [==============================] - 25s 407us/step - loss: 0.0226 - accuracy: 0.9941
Epoch 6/10
60498/60498 [==============================] - 25s 407us/step - loss: 9.4735e-04 - accuracy: 0.9998
Epoch 7/10
60498/60498 [==============================] - 25s 407us/step - loss: 1.1507e-04 - accuracy: 1.0000
Epoch 8/10
60498/60498 [==============================] - 25s 407us/step - loss: 8.4736e-05 - accuracy: 1.0000
Epoch 9/10
60498/60498 [==============================] - 25s 408us/step - loss: 6.8253e-05 - accuracy: 1.0000
Epoch 10/10
60498/604

In [70]:
predicted_classes = model.predict_classes(test_fruit_img)
accuracy_score(predicted_classes, test_label_id)

0.9798273688294055

In [71]:
#Мы добились незначительного роста точности, потеряв время на обучение

In [45]:
#Добавим в исходную архитектуру дропаут-слои
model = Sequential()
model.add(Conv2D(16, (3, 3), input_shape = (64, 64, 3), padding = "same", activation = "relu"))
model.add(Dropout(0.5))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3), padding = "same", activation = "relu"))
model.add(Dropout(0.5))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3), padding = "same", activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, (3, 3), padding = "same", activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dense(120, activation = "softmax"))

model.compile(loss = "sparse_categorical_crossentropy", optimizer = 'Adam', metrics = ['accuracy'])

In [46]:
%%time
model.fit(training_fruit_img, training_label_id, batch_size = 256, epochs = 10)

Epoch 1/10
60498/60498 [==============================] - 11s 183us/step - loss: 1.0709 - accuracy: 0.7338
Epoch 2/10
60498/60498 [==============================] - 11s 177us/step - loss: 0.0754 - accuracy: 0.9763
Epoch 3/10
60498/60498 [==============================] - 11s 177us/step - loss: 0.0347 - accuracy: 0.9898
Epoch 4/10
60498/60498 [==============================] - 11s 177us/step - loss: 0.0269 - accuracy: 0.9916
Epoch 5/10
60498/60498 [==============================] - 11s 178us/step - loss: 0.0139 - accuracy: 0.9959
Epoch 6/10
60498/60498 [==============================] - 11s 178us/step - loss: 0.0390 - accuracy: 0.9902
Epoch 7/10
60498/60498 [==============================] - 11s 178us/step - loss: 0.0020 - accuracy: 0.9996
Epoch 8/10
60498/60498 [==============================] - 11s 178us/step - loss: 0.0145 - accuracy: 0.9959
Epoch 9/10
60498/60498 [==============================] - 11s 178us/step - loss: 0.0164 - accuracy: 0.9954
Epoch 10/10
60498/60498 [============

In [47]:
predicted_classes = model.predict_classes(test_fruit_img)
accuracy_score(predicted_classes, test_label_id)

0.9380758413344972

In [ ]:
#Дропауты явно только ухудшили точность предсказания. Это свидетельствует об отсутствии переобучения в исходной модели.

In [48]:
#Упростим архитектуру сети, исключив 1 свёрточный слой.
model = Sequential()
model.add(Conv2D(16, (3, 3), input_shape = (64, 64, 3), padding = "same", activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3), padding = "same", activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3), padding = "same", activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))


model.add(Flatten())
model.add(Dense(120, activation = "softmax"))

model.compile(loss = "sparse_categorical_crossentropy", optimizer = 'Adam', metrics = ['accuracy'])

In [49]:
%%time
model.fit(training_fruit_img, training_label_id, batch_size = 256, epochs = 10)

Epoch 1/10
60498/60498 [==============================] - 8s 125us/step - loss: 0.9522 - accuracy: 0.7790
Epoch 2/10
60498/60498 [==============================] - 7s 123us/step - loss: 0.0346 - accuracy: 0.9917
Epoch 3/10
60498/60498 [==============================] - 7s 123us/step - loss: 0.0065 - accuracy: 0.9984
Epoch 4/10
60498/60498 [==============================] - 7s 124us/step - loss: 0.0161 - accuracy: 0.9955
Epoch 5/10
60498/60498 [==============================] - 7s 123us/step - loss: 0.0016 - accuracy: 0.9998
Epoch 6/10
60498/60498 [==============================] - 7s 123us/step - loss: 4.5247e-04 - accuracy: 1.0000
Epoch 7/10
60498/60498 [==============================] - 7s 122us/step - loss: 2.0120e-04 - accuracy: 1.0000
Epoch 8/10
60498/60498 [==============================] - 7s 123us/step - loss: 2.0256e-04 - accuracy: 1.00000s - loss: 2.0948e-04 - accuracy: 1. - ETA: 0s - loss: 2.0732e-04 
Epoch 9/10
60498/60498 [==============================] - 7s 122us/step - 

In [50]:
predicted_classes = model.predict_classes(test_fruit_img)
accuracy_score(predicted_classes, test_label_id)

0.9709048588885656

In [51]:
#Мы упростили архитектуру сети, что не привело к ухудшению точности на тесте. Попробуем упростить ещё сильнее

In [53]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), padding = "same", activation = "relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, (3, 3), padding = "same", activation = "relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())
model.add(Dense(120, activation = "softmax"))

model.compile(loss = "sparse_categorical_crossentropy", optimizer = 'Adam', metrics = ['accuracy'])

In [54]:
%%time
model.fit(training_fruit_img, training_label_id, batch_size = 256, epochs = 10)

Epoch 1/10
60498/60498 [==============================] - 25s 416us/step - loss: 0.3992 - accuracy: 0.9416
Epoch 2/10
60498/60498 [==============================] - 25s 406us/step - loss: 0.0016 - accuracy: 0.9996
Epoch 3/10
60498/60498 [==============================] - 24s 405us/step - loss: 0.0717 - accuracy: 0.9874
Epoch 4/10
60498/60498 [==============================] - 24s 403us/step - loss: 0.0475 - accuracy: 0.9894
Epoch 5/10
60498/60498 [==============================] - 24s 400us/step - loss: 5.6541e-04 - accuracy: 0.9999
Epoch 6/10
60498/60498 [==============================] - 25s 407us/step - loss: 0.0035 - accuracy: 0.9991
Epoch 7/10
60498/60498 [==============================] - 25s 407us/step - loss: 0.0133 - accuracy: 0.9977
Epoch 8/10
60498/60498 [==============================] - 25s 405us/step - loss: 0.0329 - accuracy: 0.9950
Epoch 9/10
60498/60498 [==============================] - 25s 405us/step - loss: 0.0112 - accuracy: 0.9977
Epoch 10/10
60498/60498 [========

In [55]:
predicted_classes = model.predict_classes(test_fruit_img)
accuracy_score(predicted_classes, test_label_id)

0.9818155368053535

In [56]:
#Точность возрасла. Проверим гипотизу о том, что в данной ситуации большую роль сыграла батч-нормализация

In [57]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), padding = "same", activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, (3, 3), padding = "same", activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())
model.add(Dense(120, activation = "softmax"))

model.compile(loss = "sparse_categorical_crossentropy", optimizer = 'Adam', metrics = ['accuracy'])

In [58]:
%%time
model.fit(training_fruit_img, training_label_id, batch_size = 256, epochs = 10)

Epoch 1/10
60498/60498 [==============================] - 10s 165us/step - loss: 0.5423 - accuracy: 0.8779
Epoch 2/10
60498/60498 [==============================] - 10s 162us/step - loss: 0.0161 - accuracy: 0.9963
Epoch 3/10
60498/60498 [==============================] - 10s 163us/step - loss: 0.0090 - accuracy: 0.9979
Epoch 4/10
60498/60498 [==============================] - 10s 163us/step - loss: 0.0188 - accuracy: 0.9951
Epoch 5/10
60498/60498 [==============================] - 10s 163us/step - loss: 4.9504e-04 - accuracy: 1.0000
Epoch 6/10
60498/60498 [==============================] - 10s 162us/step - loss: 2.3053e-04 - accuracy: 1.0000
Epoch 7/10
60498/60498 [==============================] - 10s 162us/step - loss: 1.1185e-04 - accuracy: 1.0000
Epoch 8/10
60498/60498 [==============================] - 10s 163us/step - loss: 0.0052 - accuracy: 0.9988
Epoch 9/10
60498/60498 [==============================] - 10s 163us/step - loss: 0.0384 - accuracy: 0.9905
Epoch 10/10
60498/60498 [

In [59]:
predicted_classes = model.predict_classes(test_fruit_img)
accuracy_score(predicted_classes, test_label_id)

0.9709048588885656

In [60]:
#В данном эксперименте точность предсказания без батч-нормализации упала.
#С другой стороны не следует исключать элемент случайности в первоначальной генерации весов. Эксперименты показывают,
# что при многократном повторе обучения, точность на тесте варьируется в пределах не более 0.01.
#При использовании GPU невозможно полностью исключить случайность

In [7]:
#Попробуем ещё сильнее упростить архитектуру сети.
#Батч-нормализацию берём в качестве компенсации предполагаемой потери точности.
model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape = (64, 64, 3), padding = "same", activation = "relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (4, 4)))


model.add(Flatten())
model.add(Dense(120, activation = "softmax"))

model.compile(loss = "sparse_categorical_crossentropy", optimizer = 'Adam', metrics = ['accuracy'])

In [8]:
%%time
model.fit(training_fruit_img, training_label_id, batch_size = 256, epochs = 10)

Epoch 1/10
60498/60498 [==============================] - 31s 514us/step - loss: 0.3409 - accuracy: 0.9422
Epoch 2/10
60498/60498 [==============================] - 29s 480us/step - loss: 0.0011 - accuracy: 1.0000
Epoch 3/10
60498/60498 [==============================] - 29s 479us/step - loss: 3.3477e-04 - accuracy: 1.0000
Epoch 4/10
60498/60498 [==============================] - 29s 474us/step - loss: 1.8930e-04 - accuracy: 1.0000
Epoch 5/10
60498/60498 [==============================] - 29s 473us/step - loss: 1.1985e-04 - accuracy: 1.0000
Epoch 6/10
60498/60498 [==============================] - 29s 472us/step - loss: 8.2040e-05 - accuracy: 1.0000
Epoch 7/10
60498/60498 [==============================] - 29s 476us/step - loss: 6.0732e-05 - accuracy: 1.0000
Epoch 8/10
60498/60498 [==============================] - 29s 476us/step - loss: 4.5158e-05 - accuracy: 1.0000
Epoch 9/10
60498/60498 [==============================] - 29s 472us/step - loss: 3.4258e-05 - accuracy: 1.0000
Epoch 10/

In [63]:
predicted_classes = model.predict_classes(test_fruit_img)
accuracy_score(predicted_classes, test_label_id)

0.9821549801183203

In [64]:
#На данный момент мы получили лучшее лучшую точность, используя всего 1 свёрточный слой.
#Исходя из полученных данных можно сделать вывод о том, что для данного набора изображений
#не обязательно использовать сложную многослойную архитектуру нейронной сети, которая может приводить к переобучению.
#Сохраним модель вместе с весами
model.save('best_model.h5')

In [65]:
#Сделаем заключительный тест без батч-нормализации, что позволит значительно сократить время обучения
model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape = (64, 64, 3), padding = "same", activation = "relu"))
model.add(MaxPooling2D(pool_size = (4, 4)))

model.add(Flatten())
model.add(Dense(120, activation = "softmax"))

model.compile(loss = "sparse_categorical_crossentropy", optimizer = 'Adam', metrics = ['accuracy'])

In [66]:
%%time
model.fit(training_fruit_img, training_label_id, batch_size = 256, epochs = 10)

Epoch 1/10
60498/60498 [==============================] - 10s 166us/step - loss: 0.7690 - accuracy: 0.8408
Epoch 2/10
60498/60498 [==============================] - 10s 164us/step - loss: 0.0281 - accuracy: 0.9968
Epoch 3/10
60498/60498 [==============================] - 10s 164us/step - loss: 0.0175 - accuracy: 0.9977
Epoch 4/10
60498/60498 [==============================] - 10s 164us/step - loss: 0.0047 - accuracy: 0.9998
Epoch 5/10
60498/60498 [==============================] - 10s 163us/step - loss: 0.0021 - accuracy: 1.0000
Epoch 6/10
60498/60498 [==============================] - 10s 163us/step - loss: 0.0021 - accuracy: 0.9997
Epoch 7/10
60498/60498 [==============================] - 10s 163us/step - loss: 0.0013 - accuracy: 0.9999
Epoch 8/10
60498/60498 [==============================] - 10s 164us/step - loss: 0.0172 - accuracy: 0.9961
Epoch 9/10
60498/60498 [==============================] - 10s 163us/step - loss: 9.8986e-04 - accuracy: 1.0000
Epoch 10/10
60498/60498 [========

In [67]:
predicted_classes = model.predict_classes(test_fruit_img)
accuracy_score(predicted_classes, test_label_id)

0.9713897779070895